# Demonstration of multipath fading

This demo is supposed to shed some light on the principles of [digital modulatio techniquies](https://en.wikipedia.org/wiki/Modulation#Digital_modulation_methods). 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on [communications engineering](https://en.wikipedia.org/wiki/Telecommunications_engineering) hold at the [university of applied siences, Berlin](https://www.htw-berlin.de/).

## Import libraries and set defaults

In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.style.use('noelle.mplstyle')

In [2]:
def rotate(x, y, phi):
    xr = x*np.cos(phi) + y*-np.sin(phi)
    yr = x*np.sin(phi) + y*np.cos(phi)
    
    return xr, yr

In [3]:
def plotWaves(x, y, axLim, fig, label=None):

    fig = plt.figure(fig)    
    ax = fig.add_axes((1,1,1,1), label=label)
    ax.plot(x, y)
#     print(type(ax))
#    axis([-1 1 -1 1].*axLim)
    ax.axis((-2, axLim+2, (-axLim+4)/2, (axLim+4)/2))
    ax.grid(True)
    ax.set_aspect('equal','box')

![Dreieck](https://upload.wikimedia.org/wikipedia/commons/d/dd/Dreieck.svg)

In [4]:
def plot_multipath_fading(d_tau, c, ratio_b_a, f):

#     d_tau = 2 # s
#     c = 10 # m
    d_x = 1e-3 # m
#     ratio_b_a = 1/3 #
    v = 1 # m/s
    amp = 0.3 # amp of waves

    b = ratio_b_a*(c+v*d_tau)
    a = (c+v*d_tau) - b
    B_x = c
    B_y = 0

    df = 1/((a+b)/v)  

    frequencies = np.arange(df/4,2,df/2)

    alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
    beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

    interference = np.ones_like(frequencies)
    # index of selected frequency
    f_idx = np.argmin(np.abs(frequencies-f))

    for idx, frequency in enumerate(frequencies):

        k = 2*np.pi/v*frequency # 2*pi/lambda    

        tmp_b_x, tmp_b_y = rotate(np.arange(0,b,d_x), amp*np.sin(k*np.arange(0,b,d_x)), alpha)

    #     plotWaves(w_b_x, w_b_y, c, 1)

        C_x, C_y = rotate(b, 0, alpha);
        tmp_a_x, tmp_a_y = rotate(np.arange(0,a,d_x), amp*np.sin(k*(np.arange(0,a,d_x)+b)), -beta)
        tmp_a_x = tmp_a_x + C_x
        tmp_a_y = tmp_a_y + C_y

    #     plotWaves(w_a_x, w_a_y, c, 2)

        tmp_c_x = np.arange(0,c,d_x)
        tmp_c_y = amp*np.sin(k*np.arange(0,c,d_x))
        
        interference[idx] = amp*np.sin(k*((a+b)-d_x)) + amp*np.sin(k*(c-d_x))
        
        if idx == f_idx:
            # save wave at selected frequency
            w_a_x, w_a_y, w_b_x, w_b_y, w_c_x, w_c_y = tmp_a_x, tmp_a_y, tmp_b_x, tmp_b_y, tmp_c_x, tmp_c_y            

    #     plotWaves(w_c_x, w_c_y, c, 3)


    reflector_x, reflector_y = rotate(np.arange(-1,1.1,0.1), 0, (alpha-beta)/2)
    reflector_x = reflector_x + C_x
    reflector_y = reflector_y + C_y

    n_row = 1
    n_col = 2
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)
    fig.clf()
    ax = fig.add_subplot(1,2,1)
    ax.plot(w_a_x,  w_a_y, 'C0')
    ax.plot(w_b_x,  w_b_y, 'C0')
    ax.plot(w_c_x,  w_c_y, 'C1')
    ax.axis([-2, c+2, (-c+4)/2, (c+4)/2])    
    plt.plot(reflector_x,  reflector_y, 'k')
    ax.axis('off')
    axins = inset_axes(ax, width=1.5, height=1.5)
    axins.plot(w_a_x,  w_a_y, 'C0')
    axins.plot(w_c_x,  w_c_y, 'C1')
    axins.plot(np.ones(20)*B_x, np.arange(-1,1,0.1), 'k')
    tmp_x, tmp_y = rotate(np.zeros(20), np.arange(-1,1,0.1), -beta)
    axins.plot(tmp_x+B_x, tmp_y+B_y, 'k')
    axins.plot(w_a_x[-1], w_a_y[-1], 'C0o', ms=10)
    axins.plot(w_c_x[-1], w_c_y[-1], 'C1o', ms=10)
    axins.axis([B_x-0.5, B_x+0.5, B_y-0.5, B_y+0.5])
    axins.set_xticklabels([])
    axins.set_yticklabels([])
    axins.set_xticks([])
    axins.set_yticks([])

    ax = fig.add_subplot(1,2,2)
    ax.plot(frequencies, abs(interference),ls='--', color=(0.8,0.8,0.8))
    ax.plot(f, abs(interference[f_idx]),'C3o')
    ax.axis([0, 2, 0, 1])
    ax.set_xlabel('frequency / Hz')
    ax.set_ylabel('amplitude at Rx');
    # https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Flickering-and-jumping-output
    
style = {'description_width': '30%'}
w_d_tau = widgets.FloatSlider(min=0, max=10, step=1, value=2.0, continuous_update=False, description=r'$\Delta \tau:$', style=style)
w_c = widgets.FloatSlider(min=0, max=20, step=1, value=10.0, continuous_update=False, description=r'$c:$', style=style)
w_ratio_b_a = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.33, continuous_update=False, description='ratio b/a:', style=style)
w_f = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.1, continuous_update=False, description=r'$f:$', style=style)

def update_f_slider(*args):
    v = 1
    b = w_ratio_b_a.value*(w_c.value+v*w_d_tau.value)
    a = (w_c.value+v*w_d_tau.value) - b
    df = 1/((a+b)/v)
    w_f.min = df/4
    w_f.step = df/2
    
w_ratio_b_a.observe(update_f_slider, 'value')
    
ui = widgets.HBox([w_d_tau, w_c, w_ratio_b_a, w_f])

out = widgets.interactive_output(plot_multipath_fading, {'d_tau':w_d_tau, 'c':w_c, 'ratio_b_a':w_ratio_b_a, 'f':w_f})

out.layout.height = '350px'

display(ui, out)

Output(layout=Layout(height='350px'))